# **ML MODEL CREATION AND EVALUATION**

## Objectives

* Create a classification model that can distinguish healthy from infected leaves

## Inputs

* Prepared train, validation and test datasets
  - outputs/dataset/test_X.npy
  - outputs/dataset/test_y.npy
  - outputs/dataset/validation.npy
  - outputs/dataset/validation_y.npy
  - outputs/dataset/validation_X.npy
  - outputs/dataset/validation.npy

## Outputs

* Classification model
  - outputs/model/powdery_mildew_alerter_v?.keras 

---

# Preparation for model design

## Change working directory

In [2]:
import os

os.chdir("./..")  # change to parent directory
working_dir = os.getcwd()
working_dir  # check output for correct directory

'd:\\vscode-projects\\mildew-alert'

## Load prepared data

In [3]:
import numpy as np

output_dataset = working_dir + "/outputs/dataset"
train_X = np.load(output_dataset + "/train_X.npy")
train_y = np.load(output_dataset +"/train_y.npy")
validation_X = np.load(output_dataset + "/train_X.npy")
validation_y = np.load(output_dataset +"/train_y.npy")
test_X = np.load(output_dataset + "/train_X.npy")
test_y = np.load(output_dataset +"/train_y.npy")

## Pre-processing with encoder

In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

train_y_encoded = encoder.fit_transform(train_y)
validation_y_encoded = encoder.transform(validation_y)
test_y_encoded = encoder.transform(test_y)

---

# Model

## Model design
  - input layer
  - three convolution + max pooling layers
  - flatten layer
  - two dense layers with 35% dropout between them
  - output layer
  - adaptive moment estimation optimizer and binary crossentropy loss function as suggested by empirical performance

In [ ]:
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout 

def create_model():
    model = Sequential()

    #input layer
    model.add(Input(shape=(75, 75, 3)))

    # convolution + maxpool layer 1 - 32 filters
    model.add(
        Conv2D(
            filters=32,
            kernel_size=(3,3),
            activation="relu"
               )
    )
    model.add(MaxPooling2D(2, 2))

    # convolution + maxpool layer 2 - 64 filters
    model.add(
        Conv2D(
            filters=64,
            kernel_size=(3,3),
            activation="relu"
        )
    )
    model.add(MaxPooling2D(2, 2))

    # convolution + maxpool layer 3 - 128 filters
    model.add(
        Conv2D(
            filters=128,
            kernel_size=(3,3),
            activation="relu"
        )
    )
    model.add(MaxPooling2D(2, 2))

    # flatten layer
    model.add(Flatten())

    #two dense layers with 35% dropout
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.35))
    model.add(Dense(256, activation="relu"))

    # output layer
    model.add(Dense(1, activation="sigmoid"))

    #specify optimizer, loss function and metric
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )

    return model

## Create the model

In [8]:
powdery_mildew_alerter = create_model()

## Fit data / train model

In [9]:
powdery_mildew_alerter.fit(
    train_X,
    train_y_encoded,
    epochs=7,
    verbose=1,
    validation_data=(validation_X, validation_y_encoded)
    )

Epoch 1/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 92ms/step - accuracy: 0.7415 - loss: 0.4544 - val_accuracy: 0.9908 - val_loss: 0.0447
Epoch 2/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 8s 88ms/step - accuracy: 0.9712 - loss: 0.0890 - val_accuracy: 0.9929 - val_loss: 0.0184
Epoch 3/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 9s 102ms/step - accuracy: 0.9946 - loss: 0.0229 - val_accuracy: 0.9939 - val_loss: 0.0179
Epoch 4/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 0.9971 - loss: 0.0086 - val_accuracy: 0.9487 - val_loss: 0.1337
Epoch 5/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 103ms/step - accuracy: 0.9906 - loss: 0.0401 - val_accuracy: 0.9983 - val_loss: 0.0046
Epoch 6/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 105ms/step - accuracy: 0.9968 - loss: 0.0129 - val_accuracy: 0.9997 - val_loss: 9.9291e-04
Epoch 7/7
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - accuracy: 1.0000 - loss: 4.5322e-04 - val_accuracy: 1.0000 - val_loss: 1.8276e-04


## Evaluate performance

In [10]:
powdery_mildew_alerter.evaluate(test_X, test_y_encoded, verbose=1)

92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 1.0000 - loss: 2.0336e-04


[0.00018275903130415827, 1.0]

Model has a 100% accuracy on validation and test sets

## Save model (change version variable if needed)

In [12]:
version = "v1"
save_dir = working_dir + "/outputs/model"

if not "model" in os.listdir(working_dir + "/outputs"):
    os.makedirs(save_dir)

powdery_mildew_alerter.save(f"{save_dir}/powdery_mildew_alerter_{version}.keras")

---

# Generate model evaluation visuals

---